In [2]:
import os

In [3]:
%pwd

'c:\\Mlops-projects\\project-1\\datascienceproject\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Mlops-projects\\project-1\\datascienceproject'

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [7]:
@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories

In [14]:
class ConfigurationManager:
    def __init__(self,
                config_filepath=CONFIG_FILE_PATH,
                params_filepath=PARAMS_FILE_PATH,
                schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [15]:
import urllib.request as request
from src.datascience import logger
import zipfile

In [11]:
###component DataIngestion
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config
    
    ##downloading the zip file
    def download_file(self):
        
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info:\n{headers}")
        else:
            logger.info(f"file already exists")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted files to {unzip_path}")


In [16]:
try:
    config=ConfigurationManager()
    data_ingetion_config=config.get_data_ingestion_config()
    data_ingetion=DataIngestion(config=data_ingetion_config)
    data_ingetion.download_file()
    data_ingetion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-10 10:59:52,258: INFO: common: yaml file  from these path config\config.yaml successfully loaded]
[2025-01-10 10:59:52,262: INFO: common: yaml file  from these path params.yaml successfully loaded]
[2025-01-10 10:59:52,263: INFO: common: yaml file  from these path schema.yaml successfully loaded]
[2025-01-10 10:59:52,267: INFO: common: successfully created directory at :artifacts]
[2025-01-10 10:59:52,267: INFO: common: successfully created directory at :artifacts/data_ingestion]
[2025-01-10 10:59:54,398: INFO: 2844951490: artifacts/data_ingestion/data.zip download! with following info:
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-